### 인스타그램 해시태그 크롤링
- 셀레니움을 이용한 크롤링
- 결과에 대한 단어 분석

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs 
import pandas as pd
import re
import time
import sys

In [2]:
# 화면 아래쪽으로 스크롤
def page_Down():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(2)

### 1. 인스타그램 로그인

In [12]:
# 드라이버 실행
driver = Chrome("./chromedriver")
url = "instagram"
# 인스타그램 접속
driver.get("https://www.instagram.com/")
time.sleep(2)

In [33]:
# 인스타그램 로그인
userID = "아이디"
userPW = "비밀번호"

# driver.find_element_by_css_selector("#loginForm > div > div:nth-child(1) > div > label > input").click()
input_id = driver.find_element(By.CSS_SELECTOR," #loginForm > div > div:nth-child(1) > div > label > input")
input_id.clear()
input_id.send_keys(userID)

input_pw = driver.find_element(By.CSS_SELECTOR, "#logihnForm > div > div:nth-child(2) > div > label > input")
input_pw.clear()
input_pw.send_keys(userPW)

input_pw.submit() # 데이터 전송
time.sleep(3)

driver.find_element(By.CSS_SELECTOR, "#react-root > section > main > div > div > div > div > button").click()


#### 2. 인스타그램에서 검색 url로 접속하기

In [39]:
# 인스타 검색 url 확인 및 사용자 키워드 적용
keyword = input("검색어 입력:")
url = "https://www.instagram.com/explore/tags/" + keyword
driver.get(url)

검색어 입력:제주도맛집


#### 3. 게시글 가져오기

In [40]:
# 첫번째 게시물 클릭
driver.find_element(By.CSS_SELECTOR, "div._9AhH0").click()
time.sleep(3)

In [61]:
# 첫번째 게시물 정보 가져오기
html = driver.page_source
soup = bs(html, "html.parser")
# print(soup)

# content = soup.select("div.C4VMK > div.MOdxS > span")
content = soup.find("div", class_="C4VMK").find_all("span")[1].text
content

'<협찬>재철 식재료로 매일 신선한 스시,숙성회를 맛보실수 있는 스시오타루🏠제주 제주시 인다2길 30🏠제주 제주시 아라일동 6117-8🕓매일 11:30 - 22:00(브레이크타임 15:00 - 17:00)☎064-752-5271🌟포장가능🌟스시오타루#제주맛집 #제주도맛집 #제주시맛집 #제주오마카세 #제주도오마카세#제주스시 #아라동맛집 #연동맛집#제주시청맛집 #신제주맛집 #구제주맛집#제주도시락 #제주도도시락 #제주회포장#제주초밥 #제주도초밥'

In [186]:
# 다음 게시물로 이동
# driver.find_element(By.CSS_SELECTOR, "body > div.RnEpo._Yhr4 > div.Z2Inc._7c9RR > div > div.l8mY4.feth3 > button > div > span > svg").click()
driver.find_element(By.XPATH, "/html/body/div[6]/div[2]/div/div/button").click()
# time.sleep(3)


In [183]:
# 두번째 이후 게시물 이동
driver.find_element(By.XPATH, "/html/body/div[6]/div[2]/div/div[2]/button").click()

In [191]:
def get_first(driver):
    driver.find_element(By.CSS_SELECTOR, "div._9AhH0").click()
    time.sleep(3)

In [211]:
def get_content(driver):
    html = driver.page_source
    soup = bs(html, "html.parser")
    
    content = soup.find("div", class_="C4VMK").find_all("span")[1].text
    if content.split("#")[0][:4] == "<협찬>" or content.split("#")[0][:4] == "<광고>":
        pass
    else:
        print(content.split("#"[0]))  # 내용만 출력하기
    
    
    # 협찬/광고가 아닌 데이터만 출력하기
    content_txt = content.split("#"[0])
    print(content_txt[:4])
    print()

In [203]:
def get_next(driver, cnt):
    if cnt == 0:
        driver.find_element(By.XPATH, "/html/body/div[6]/div[2]/div/div/button").click()
        time.sleep(3)
    else:
        driver.find_element(By.XPATH, "/html/body/div[6]/div[2]/div/div[2]/button").click()
        time.sleep(3)

In [212]:
# 인스타 검색 url 확인 및 사용자 키워드 적용
keyword = input("검색어 입력:")
url = "https://www.instagram.com/explore/tags/" + keyword
driver.get(url)
time.sleep(7)

get_first(driver)
for i in range(5):
    get_content(driver)
    get_next(driver, i)

검색어 입력:제주도맛집
['<협찬>재철 식재료로 매일 신선한 스시,숙성회를 맛보실수 있는 스시오타루🏠제주 제주시 인다2길 30🏠제주 제주시 아라일동 6117-8🕓매일 11:30 - 22:00(브레이크타임 15:00 - 17:00)☎064-752-5271🌟포장가능🌟스시오타루', '제주맛집 ', '제주도맛집 ', '제주시맛집 ']

['', '제주도맛집🍜 제주도민맛집💯 제주순메밀막국수🍜🎐 100% 제주산 순메밀로만 만들어낸🎐 입에 넣는순간 차이을 알게되는🎐 제주도민도 인정하는 전통깊은 맛집🎈..', '카멜리아힐맛집.[20kg당 한포대 5만원] 밀가루 70% 수입메밀 30% 를 쓰는 일반 가게와는 다르게🔺저희 제주순메밀막국수에서는 [20kg당 25만원] 🍜국내산 100% 순메밀🍜을 사용하기 때문에 더욱 진하고 건강한 맛을 느끼실 수 있습니다😋저희 순메밀막국수는 코로나방역지침을 준수하여 영업하고 있습니다💖____________________________________주소 : 제주 서귀포시 안덕면 서광리 1413-3전화 : 064-792-0600____________________________________', '안덕면맛집', '제주도서쪽맛집 ', '제주국제학교맛집 ', '오설록근처맛집 ', '제주도오설록맛집', '제주도국수']
['', '제주도맛집🍜 제주도민맛집💯 제주순메밀막국수🍜🎐 100% 제주산 순메밀로만 만들어낸🎐 입에 넣는순간 차이을 알게되는🎐 제주도민도 인정하는 전통깊은 맛집🎈..', '카멜리아힐맛집.[20kg당 한포대 5만원] 밀가루 70% 수입메밀 30% 를 쓰는 일반 가게와는 다르게🔺저희 제주순메밀막국수에서는 [20kg당 25만원] 🍜국내산 100% 순메밀🍜을 사용하기 때문에 더욱 진하고 건강한 맛을 느끼실 수 있습니다😋저희 순메밀막국수는 코로나방역지침을 준수하여 영업하고 있습니다💖____________________________________주소 : 제주 서귀포시 안덕면 서광리 1413-3전화 : 064-792-0600_______________

In [294]:
keyword = input("검색어 입력:")
cntdown = int(input("스크롤 다운 건수:"))

driver=Chrome("./chromedriver")

# 인스타그램 접속
driver.get("https://www.instagram.com/")
time.sleep(2)

# 인스타그램 로그인
userID="아이디"
userPW="비밀번호"

# 아이디 입력
input_id=driver.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(1) > div > label > input")
input_id.send_keys(userID)

# 패스워드 입력
input_pw=driver.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(2) > div > label > input")
input_pw.send_keys(userPW)

# 로그인 정보 전송 / 로그인 버튼을 클릭해도 됨
input_pw.submit()  
time.sleep(3)

state = input("로그인 되었습니까?[y/n]:")
if state == "n" or state == "ㅜ" or state == "N":
    print("작업을 종료합니다.")
    driver.close()
    sys.exit()

검색어 입력:제주도맛집
스크롤 다운 건수:1
로그인 되었습니까?[y/n]:y


In [295]:
# 인스타 검색 url 확인 및 사용자 키워드 적용

url="https://www.instagram.com/explore/tags/" + keyword
driver.get(url)
time.sleep(5)

body = driver.find_element(By.TAG_NAME, "body")

In [296]:
# 하이퍼 링크 호출
link_url = []

for down in range(cntdown):
    page_Down()
    contents = body.find_elements(By.CLASS_NAME, "v1Nh3")
    
    # 게시글 리스트 URL 추출
    for content in contents:
        url = content.find_element(By.TAG_NAME, "a").get_attribute("href")
        link_url.append(url)
        
link_url

['https://www.instagram.com/p/CbrJDVqAt2S/',
 'https://www.instagram.com/p/CbqxeCshE2y/',
 'https://www.instagram.com/p/Cboikw_DONk/',
 'https://www.instagram.com/p/CbqxeEBBeA5/',
 'https://www.instagram.com/p/CbojUN_hnHm/',
 'https://www.instagram.com/p/CbqxeCvhPGD/',
 'https://www.instagram.com/p/CbqsybIhLct/',
 'https://www.instagram.com/p/CbrCnDepiCL/',
 'https://www.instagram.com/p/CboUX-2rINL/',
 'https://www.instagram.com/p/CbrTrhlrT6z/',
 'https://www.instagram.com/p/CbrTkT0rBxh/',
 'https://www.instagram.com/p/CbrTq9pvupy/',
 'https://www.instagram.com/p/CbrTVZ-vxGu/',
 'https://www.instagram.com/p/CbrTZrTrItE/',
 'https://www.instagram.com/p/CbrTfyOr_vM/',
 'https://www.instagram.com/p/CbrS2Z-vpwW/',
 'https://www.instagram.com/p/CbrS61cL_Hf/',
 'https://www.instagram.com/p/CbrS4g-L0j3/',
 'https://www.instagram.com/p/CbrSr5XLSus/',
 'https://www.instagram.com/p/CbrSnSyrSZF/',
 'https://www.instagram.com/p/CbrSoMUvLvM/',
 'https://www.instagram.com/p/CbrSVLGL98U/',
 'https://

In [297]:
contents = body.find_elements(By.CLASS_NAME, "v1Nh3")
contents[0].find_element(By.TAG_NAME, "a").get_attribute("href")

'https://www.instagram.com/p/CbrJDVqAt2S/'

In [298]:
link_url=list(set(link_url)) # 중복 데이터 제거
conttxt = []
for i in range(5):
    driver.get(link_url[i])
    time.sleep(2)
    
    html = driver.page_source
    soup = bs(html, "html.parser")
    conttxt.append(soup.find("div", class_="C4VMK").find_all("span")[1].text)

In [299]:
re.sub('[^0-9a-zA-Zㄱ-ㅣ가-힣#?! ]',"",conttxt[0])

'Purple Moss VintageRalph Lauren Hoodie Zipup unisexSize  M  L 남성기준기장  64cm소매  64cm가슴  57cm어깨  48cm위 실측을 꼭 참고해주세요!!모든 문의는 DM으로 문의주세요1200  20002100Mon  Sun 연중무휴띄제주시 애월읍 하귀로 53수입빈티지샵 퍼플모스Country Origin  EU  US  ASIA#퍼플모스구매삽가능'

In [300]:
# 아이디 가져오기
soup.find("div", class_="C4VMK").find("a").get_text()

'jeju_insaeng'

In [301]:
# 본문 가져오기
soup.find('div',class_="MOdxS").find('span').text.split("#")[0]

'제주 인생밥집⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀🕚 OPEN - 11:00⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀🕕 CLOSED - 17:00⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀브레이크타임 없어요.⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀예약은 받지 않고 오시는 순서대로 식사가능 하세요.⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀싱싱한 해물과 신선한 야채로 그날 준비한 재료가 소진되면 조기마감 합니다. ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀🥣전복해물볶음밥 / 전복물회 / 전복딱새우장 정식⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀사는게니나노 ~ ♬인생밥집 주소는 제주시 한림읍 문교길6 ..⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀🅿️ 무료 공영 주차장 위치!! .⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀🚗 제주시 한림읍 한림리 1197-4🚘 제주시 한림읍 한림리 1198-3🚙 제주시 한림읍 한림리 943-9.걸어서 2분이내 거리에요~♡♡'

In [302]:
# 해시태그 가져오기
tags = soup.find('div',class_="MOdxS").find_all('a')
for tag in tags:
    if "#" in tag.text:
        print(tag.text)

#제주
#제주도
#jeju
#제주살이
#제주맛집
#제주도맛집
#한림맛집
#애월맛집
#대구
#중문맛집
#산방산맛집
#월정리
#신화월드맛집
#제주신혼여행
#협재해수욕장근처맛집
#협재해변
#금능해수욕장
#좋아요
#일상
#일상스타그램
#먹스타그램
#맛스타그램
#전복장
#딱새우장
#제주가족여행
#제주우정여행
#제주혼밥
#협재혼밥
